In [1]:
import requests
import json

In [ ]:
get details of GH repos and insert it to DynamoDB table

In [29]:
#get repo id
res = requests.get(
    'https://api.github.com/repos/AverbakhMV/From-GH-to-DynamoDB',
    headers={'Authorization': f'token {token}'}
)
repo = json.loads(res.content.decode('utf-8'))
repo

{'id': 651047891,
 'node_id': 'R_kgDOJs4z0w',
 'name': 'From-GH-to-DynamoDB',
 'full_name': 'AverbakhMV/From-GH-to-DynamoDB',
 'private': False,
 'owner': {'login': 'AverbakhMV',
  'id': 87974490,
  'node_id': 'MDQ6VXNlcjg3OTc0NDkw',
  'avatar_url': 'https://avatars.githubusercontent.com/u/87974490?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/AverbakhMV',
  'html_url': 'https://github.com/AverbakhMV',
  'followers_url': 'https://api.github.com/users/AverbakhMV/followers',
  'following_url': 'https://api.github.com/users/AverbakhMV/following{/other_user}',
  'gists_url': 'https://api.github.com/users/AverbakhMV/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/AverbakhMV/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/AverbakhMV/subscriptions',
  'organizations_url': 'https://api.github.com/users/AverbakhMV/orgs',
  'repos_url': 'https://api.github.com/users/AverbakhMV/repos',
  'events_url': 'https://api.github.com/users

In [3]:
def list_repos(token, since='651047891'):
    res = requests.get(
        f'https://api.github.com/repositories?since={since}',
        headers={'Authorization': f'token {token}'}
    )
    return json.loads(res.content.decode('utf-8'))

In [4]:
repos = list_repos(token)

In [5]:
def get_repo_details(owner, name, token):
    repo_details = json.loads(requests.get(
        f'https://api.github.com/repos/{owner}/{name}',
        headers={'Authorization': f'token {token}'}
    ).content.decode('utf-8'))
    return repo_details

In [7]:
def extract_repo_fields(repo_details):
    repo_fields = {
        'id': repo_details['id'],
        'node_id': repo_details['node_id'],
        'name': repo_details['name'],
        'full_name': repo_details['full_name'],
        'owner': {
            'login': repo_details['owner']['login'],
            'id': repo_details['owner']['id'],
            'node_id': repo_details['owner']['node_id'],
            'type': repo_details['owner']['type'],
            'site_admin': repo_details['owner']['site_admin']
        },
        'html_url': repo_details['html_url'],
        'description': repo_details['description'],
        'fork': repo_details['fork'],
        'created_at': repo_details['created_at']
    }
    return repo_fields

In [8]:
def get_repos(repos, token):
    repos_details = []
    for repo in repos:
        try:
            owner = repo['owner']['login']
            name = repo['name']
            repo_details = get_repo_details(owner, name, token)
            repo_fields = extract_repo_fields(repo_details)
            repos_details.append(repo_fields)
        except:
            pass
    return repos_details

In [10]:
repos_details = get_repos(repos, token)

In [11]:
len(repos_details)

99

In [12]:
import boto3

In [13]:
import os

In [14]:
os.environ.setdefault('AWS_PROFILE', 'GitHubUser')

'GitHubUser'

In [15]:
os.environ.setdefault('AWS_DEFAULT_REGION', 'us-east-1')

'us-east-1'

In [26]:
#create dynamoDB table
ghrepos = dynamodb.create_table(
    TableName='ghrepos',
    KeySchema=[
        {
            'AttributeName': 'id',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'id',
            'AttributeType': 'N'
        }
    ],
    BillingMode='PAY_PER_REQUEST'
)       

In [25]:
%%sh
aws dynamodb list-tables --profile GitHubUser --region us-east-1

{
    "TableNames": [
        "ghmarker",
        "ghrepos"
    ]
}


In [16]:
dynamodb = boto3.resource('dynamodb')

In [17]:
ghrepos_table = dynamodb.Table('ghrepos')

In [18]:
ghrepos_table.item_count

0

In [19]:
def load_repos(repos_details, ghrepos_table):
    for repo in repos_details:
        ghrepos_table.put_item(Item=repo)

In [20]:
%%time
load_repos(repos_details, ghrepos_table)

CPU times: user 460 ms, sys: 7.12 ms, total: 468 ms
Wall time: 26.2 s


In [21]:
items = ghrepos_table.scan()

In [23]:
len(items['Items'])

99

In [24]:
items['Items'][0]

{'created_at': '2023-06-08T11:28:48Z',
 'owner': {'site_admin': False,
  'id': Decimal('52549054'),
  'login': 'daniel-w-f',
  'type': 'User',
  'node_id': 'MDQ6VXNlcjUyNTQ5MDU0'},
 'full_name': 'daniel-w-f/CGAD',
 'html_url': 'https://github.com/daniel-w-f/CGAD',
 'description': 'CGAD = ChatGPT API Dings',
 'id': Decimal('651048037'),
 'fork': False,
 'name': 'CGAD',
 'node_id': 'R_kgDOJs40ZQ'}